# Monte Carlo simulation of rock properties


We can easily draw randomly from distributions of properties:

- Normal: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.normal.html
- Uniform: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.uniform.html
- Lognormal: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.lognormal.html

The normal distribution is probably familiar:

<img src="https://subsurfwiki.org/images/3/3a/Normal_distribution.png" width="500px" />

In [ ]:
import numpy as np

np.random.seed(42)

rho = np.random.normal(loc=2500, scale=125, size=200)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(rho)

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")

sns.displot(rho, rug=True, kde=True)

## Same thing in `scipy`

There are continuous (and discrete) distributions in `scipy` too. There are more of them ([a lot more!](https://docs.scipy.org/doc/scipy/reference/tutorial/stats/continuous.html#continuous-distributions-in-scipy-stats)), and they allow a bit more flexibility.

In [ ]:
import scipy.stats as st

mean = 3   # aka mu or loc
stdev = 2  # aka sigma or scale
normal_distribution = st.norm(loc=mean, scale=stdev)

x = np.linspace(-6, 12, 200)

_, ax = plt.subplots()
ax.plot(x, normal_distribution.pdf(x), '-', lw=2)
plt.title(f'Normal distribution: mean = {mean}, stdev = {stdev}')
plt.grid()
plt.show()

In [ ]:
d_rho = st.norm(loc=2500, scale=125)
d_rho.rvs()  # Random variates

In [ ]:
rho = d_rho.rvs(size=200, random_state=42)

sns.displot(rho)

## Fit a distribution

Using the Rock Property Catalog: https://subsurfwiki.org/wiki/Rock_Property_Catalog

In [ ]:
import pandas as pd

df = pd.read_csv('https://geocomp.s3.amazonaws.com/data/RPC_4_lithologies.csv')
df = df.dropna()

df.head()

#### NOTE

The original density data (`df.Rho`) were discretized in the original lab measurements. I've added some random noise to these values to get a more natural distribution. So we'll use `df.Rho_n`.

In [ ]:
for name, group in df.groupby('Lithology'):
    plt.scatter(group.Vp, group.Rho_n, label=name)
plt.legend()

In [ ]:
for name, group in df.groupby('Lithology'):
    sns.kdeplot(group.Vp, label=name)
plt.legend()

Let's fit a normal distribution to this data. We'll focus on limestone first.

In [ ]:
from scipy.stats import norm

limestone = df.loc[df.Lithology=='limestone']

loc, scale = st.norm.fit(limestone.Vp)

In [ ]:
loc, scale

Instantiate the learned distribution:

In [ ]:
l_vp = st.norm(loc=loc, scale=scale)

Compare its PDF to the actual distribution:

In [ ]:
x = np.linspace(1500, 5000, 1000)

sns.kdeplot(limestone.Vp)
plt.plot(x, l_vp.pdf(x))

Maybe a skewed distribution, like `gumbel_r` (`r` for 'right') distribution is better:

In [ ]:
loc, scale = st.gumbel_r.fit(limestone.Vp)

l_vp = st.gumbel_r(loc=loc, scale=scale)

sns.kdeplot(limestone.Vp)
plt.plot(x, l_vp.pdf(x))

Sample from the distribution:

In [ ]:
vp = l_vp.rvs(size=200)

sns.displot(vp, rug=True)

### EXERCISE

Repeat this exercise for the `df.Rho_n` data. You should end up with an array of 200 samples drawn from the distribution.

If there's time, check some other distributions in `scipy.stats`.

In [ ]:
# YOUR CODE HERE



In [ ]:
# Check the distribution:
sns.kdeplot(limestone.Rho_n)

In [ ]:
loc, scale = st.norm.fit(limestone.Rho_n)
l_rho = st.norm(loc=loc, scale=scale)

# Do in one step:
#   d_rho = norm(*norm.fit(dolomite.Rho_n))

rho = l_rho.rvs(size=200)

sns.displot(rho, kde=True)

## Kernel density estimation

It's possible to model 'lumpy' distributions as 'mixtures of Gaussians', the most general expression of which is the kernel density estimate. 

Let's model the limestone's `Rho_n` distribution that way.

In [ ]:
sns.kdeplot(limestone.Rho_n)

Unfortunately, there's no easy way to get the KDE that Seaborn is plotting. We'll have to compute it ourselves.

We could use `scipy.stats.gaussian_kde()` but it doesn't have a way to generate ranvom variates, whereas `KernelDensity` in `sklearn.neighbors` comes with a `sample()` method, so let's use that.

In [ ]:
from sklearn.neighbors import KernelDensity

X = np.array(limestone.Rho_n).reshape(-1, 1)  # X must be 2D.
kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(X)
sample = np.squeeze(kde.sample(n_samples=500))

_ = plt.hist(sample, bins=20)

Nice!

## Joint distributions

Sometimes variables vary together, so we can't draw them separately.

Let's simulate impedance!

In [ ]:
def impedance(vp, rho):
    return vp * rho

imp = impedance(vp, rho)

imp.shape

In [ ]:
sns.displot(imp, kde=True)

Let's look at the joint distribution:

In [ ]:
sns.jointplot(x=vp, y=rho)

And compare this to the actual data:

In [ ]:
sns.jointplot(x=dolomite.Vp, y=dolomite.Rho_n)

We'll have to do something about that.

### Multivariate Gaussian simulation

If the two distributions are Gaussian, then we can use a multivariate Gaussian distribution:

In [ ]:
mean = np.mean(df[['Vp', 'Rho_n']])
cov = np.cov(df[['Vp', 'Rho_n']], rowvar=False)  # vars are in columns.

multi = st.multivariate_normal(mean=mean, cov=cov, seed=42)

samples = multi.rvs(size=200, random_state=42)

samples.shape

In [ ]:
vp, rho = samples.T

sns.jointplot(x=vp, y=rho)

Problem solved!

But... this multivariate Gaussian simulation only works on normally distributed variables. What if our variables do not fit normal distributions? 

We will need to transform them to Gaussians, simulate them, then back-transform them to their original distribution.

### What if the marginal distributions are not Gaussian?

So we have a way to model normal distributions; but what if your variables are better modeled by other distributions? 

We can transform random variables to and from a uniform distribution using the [probabiity integral transform](https://en.wikipedia.org/wiki/Probability_integral_transform). From Wikipedia: 

> [...] the probability integral transform (also known as universality of the uniform) relates to the result that data values that are modeled as being random variables from any given continuous distribution can be converted to random variables having a standard uniform distribution. This holds exactly provided that the distribution being used is the true distribution of the random variables; if the distribution is one fitted to the data, the result will hold approximately in large samples.

I'm following [this blog post](https://twiecki.io/blog/2018/05/03/copulas/).

Let's start with a uniform sampling on [0, 1]:

In [ ]:
x = st.uniform(0, 1).rvs(10000)
sns.displot(x, kde=False)

We can transform these samples to a normal distribution:

In [ ]:
norm = st.norm()
x_trans = norm.ppf(x)
sns.displot(x_trans)

We can visualize the inverse CDF given by the model's `ppf()` method:

In [ ]:
h = sns.jointplot(x=x, y=x_trans)
h.set_axis_labels('original', 'transformed', fontsize=16)

We can do this with any distribution:

In [ ]:
gumbel = st.gumbel_l()
x_trans = gumbel.ppf(x)
h = sns.jointplot(x=x, y=x_trans)
h.set_axis_labels('original', 'transformed', fontsize=16)

To go backwards, we apply the CDF (the inverse of the inverse CDF!)...

In [ ]:
x_trans_trans = gumbel.cdf(x_trans)
h = sns.jointplot(x=x_trans, y=x_trans_trans)
h.set_axis_labels('original', 'transformed', fontsize=16)

This means we can convert a Gumbel (say) to a Gaussian:

In [ ]:
# First we have to make the Gumbel we're pretending we're starting from:
gumbel = st.gumbel_l()
x_gumbel = gumbel.ppf(x)

# Now transform to uniform, then Gaussian:
x_gumbel_uniform = gumbel.cdf(x_trans)

norm = st.norm()
x_uniform_normal = norm.ppf(x_gumbel_uniform)

# And plot:
h = sns.jointplot(x=x_gumbel, y=x_uniform_normal)
h.set_axis_labels('original', 'transformed', fontsize=16)

Why would we do this? From [a blog post by Pyrcz and Deutsch](http://www.geostatisticslessons.com/lessons/normalscore)

> Modern geostatistical algorithms and software all invoke the multivariate Gaussian (MG) distribution for probabilistic prediction of continuous properties. A requirement of the MG distribution is that the univariate distribution must be Gaussian. The procedure developed early on in multivariate statistics and adopted by geostatistics is to: (1) transform the data to a univariate Gaussian distribution, (2) proceed with algorithms that take advantage of the properties of the multivariate Gaussian distribution, then (3) back transform results to original units.

## Another dataset

Let's look at some porosity data. We'll use data from Atkinson, CD, JH McGowen, S Bloch, LL Lundell, and PN Trumbly, 1990, Braidplain and deltaic reservoir, Prudhoe Bay, Alaska, in JH Barwis, JG McPherson, and RJ Studlick, eds, Sandstone petroleum reservoirs: New York, Springer-Verlag, p 7–29.

In [ ]:
import pandas as pd

# Read a file from Google Sheets:
uid = "1QcSw_xRAYgJzD9HsIXNjmS7o4Zb6qkRBgIWhmp4f2mI"
url = f"https://docs.google.com/spreadsheets/d/{uid}/export?format=csv"
df = pd.read_csv(url)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for name, group in df.groupby('Gross environment'):
    plt.scatter(group.Porosity, np.log10(group.Permeability), label=name)
plt.legend()

In [ ]:
deltaic = df.loc[df['Gross environment']=='Deltaic']

sns.displot(deltaic['Porosity'], kde=True)

### EXERCISE

Can you create a KDE of this porosity data and draw 1000 samples from it?

Can you create a joint poro-perm distribution for the Deltaic environment, and draw 1000 samples from that?

In [ ]:
# YOUR CODE HERE



In [ ]:
# Solution!




---

## All the distributions!

You could even compute all the distributions!

In [ ]:
# From: https://stackoverflow.com/a/37616966/3381305
# (c): User stackoverflow.com/users/2087463/tmthydvnprt
# Licensed: CC-BY-SA 4.0

import warnings
import matplotlib
from scipy.stats._continuous_distns import _distn_names
from tqdm import tqdm

def best_fit_distribution(data, bins=100, ax=None, skip=None):
    """Model data by finding best fit distribution to data"""
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0
    
    if skip is None:
        skip = []

    DISTRIBUTIONS = [getattr(st, d) for d in _distn_names if d not in skip]

    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    for distribution in tqdm(DISTRIBUTIONS):

        try:
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                params = distribution.fit(data)
                
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                except Exception:
                    pass

                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return best_distribution.name, best_params

def make_pdf(dist, params, size=1000):
    """Generate distributions's Probability Distribution Function """

    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

data = df.Vp

# Make the "All dists" plot.
plt.figure(figsize=(10, 6))
ax = data.plot(kind='hist', bins=20, density=True, alpha=0.5)
dataYLim = ax.get_ylim()

best_fit_name, best_fit_params = best_fit_distribution(data, 20, ax, skip=['levy_stable'])
best_dist = getattr(st, best_fit_name)

ax.set_ylim(dataYLim)
ax.set_title('All fitted distributions')
ax.set_xlabel('Vp')
ax.set_ylabel('Frequency')

# Make plot with best params 
pdf = make_pdf(best_dist, best_fit_params)

plt.figure(figsize=(10, 6))
ax = pdf.plot(lw=2, label='PDF', legend=True)
data.plot(kind='hist', bins=20, normed=True, alpha=0.5, label='Data', legend=True, ax=ax)

param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_fit_params)])
dist_str = '{}({})'.format(best_fit_name, param_str)

ax.set_title('Best fit distribution\n' + dist_str)
ax.set_xlabel('Vp')
ax.set_ylabel('Frequency')

plt.show()

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2020</p>
</div>